In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import joblib
from sklearn.preprocessing import StandardScaler, OneHotEncoder

current_dir = os.getcwd()
src_dir = os.path.dirname(os.path.dirname(current_dir))
project_root = os.path.dirname(src_dir)

if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
from src.helpers.file_utils import FileUtils
from src.helpers.logger import info

# Ustawienia wyświetlania w pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# --- KROK 1: Konfiguracja Eksperymentu ---

EXPERIMENT_VERSION = 0 # Wybierz 0, 1 lub 2
columns_to_keep_1 = [
    'M_DATE', 'RM_PX_1', 'RM_PX_1_FSQ', 'RM_PX_1_R', 'RM_PX_1_POS', 'RM_PX_1_RT_M', 'RM_PX_1__RT_PS', 'RM_PX_1_FORM5', 'RM_PX_1_WINR', 'RM_PX_1_G90', 'RM_PX_1_A90', 'RM_PX_1_KP9', 
    'RM_PX_2', 'RM_PX_2_FSQ', 'RM_PX_2_R', 'RM_PX_2_POS', 'RM_PX_2_RT_M', 'RM_PX_2__RT_PS', 'RM_PX_2_FORM5', 'RM_PX_2_WINR', 'RM_PX_2_G90', 'RM_PX_2_A90', 'RM_PX_2_KP9', 
    'RM_PX_3', 'RM_PX_3_FSQ', 'RM_PX_3_R', 'RM_PX_3_POS', 'RM_PX_3_RT_M', 'RM_PX_3__RT_PS', 'RM_PX_3_FORM5', 'RM_PX_3_WINR', 'RM_PX_3_G90', 'RM_PX_3_A90', 'RM_PX_3_KP9', 
    'RM_PX_4', 'RM_PX_4_FSQ', 'RM_PX_4_R', 'RM_PX_4_POS', 'RM_PX_4_RT_M', 'RM_PX_4__RT_PS', 'RM_PX_4_FORM5', 'RM_PX_4_WINR', 'RM_PX_4_G90', 'RM_PX_4_A90', 'RM_PX_4_KP9', 
    'RM_PX_5', 'RM_PX_5_FSQ', 'RM_PX_5_R', 'RM_PX_5_POS', 'RM_PX_5_RT_M', 'RM_PX_5__RT_PS', 'RM_PX_5_FORM5', 'RM_PX_5_WINR', 'RM_PX_5_G90', 'RM_PX_5_A90', 'RM_PX_5_KP9', 
    'RM_PX_6', 'RM_PX_6_FSQ', 'RM_PX_6_R', 'RM_PX_6_POS', 'RM_PX_6_RT_M', 'RM_PX_6__RT_PS', 'RM_PX_6_FORM5', 'RM_PX_6_WINR', 'RM_PX_6_G90', 'RM_PX_6_A90', 'RM_PX_6_KP9', 
    'RM_PX_7', 'RM_PX_7_FSQ', 'RM_PX_7_R', 'RM_PX_7_POS', 'RM_PX_7_RT_M', 'RM_PX_7__RT_PS', 'RM_PX_7_FORM5', 'RM_PX_7_WINR', 'RM_PX_7_G90', 'RM_PX_7_A90', 'RM_PX_7_KP9', 
    'RM_PX_8', 'RM_PX_8_FSQ', 'RM_PX_8_R', 'RM_PX_8_POS', 'RM_PX_8_RT_M', 'RM_PX_8__RT_PS', 'RM_PX_8_FORM5', 'RM_PX_8_WINR', 'RM_PX_8_G90', 'RM_PX_8_A90', 'RM_PX_8_KP9', 
    'RM_PX_9', 'RM_PX_9_FSQ', 'RM_PX_9_R', 'RM_PX_9_POS', 'RM_PX_9_RT_M', 'RM_PX_9__RT_PS', 'RM_PX_9_FORM5', 'RM_PX_9_WINR', 'RM_PX_9_G90', 'RM_PX_9_A90', 'RM_PX_9_KP9', 
    'RM_PX_10', 'RM_PX_10_FSQ', 'RM_PX_10_R', 'RM_PX_10_POS', 'RM_PX_10_RT_M', 'RM_PX_10__RT_PS', 'RM_PX_10_FORM5', 'RM_PX_10_WINR', 'RM_PX_10_G90', 'RM_PX_10_A90', 'RM_PX_10_KP9', 
    'RM_PX_11', 'RM_PX_11_FSQ', 'RM_PX_11_R', 'RM_PX_11_POS', 'RM_PX_11_RT_M', 'RM_PX_11__RT_PS', 'RM_PX_11_FORM5', 'RM_PX_11_WINR', 'RM_PX_11_G90', 'RM_PX_11_A90', 'RM_PX_11_KP9', 
    'RM_PX_12', 'RM_PX_12_FSQ', 'RM_PX_12_R', 'RM_PX_12_POS', 'RM_PX_12_RT_M', 'RM_PX_12__RT_PS', 'RM_PX_12_FORM5', 'RM_PX_12_WINR', 'RM_PX_12_G90', 'RM_PX_12_A90', 'RM_PX_12_KP9', 
    'RM_PX_13', 'RM_PX_13_FSQ', 'RM_PX_13_R', 'RM_PX_13_POS', 'RM_PX_13_RT_M', 'RM_PX_13__RT_PS', 'RM_PX_13_FORM5', 'RM_PX_13_WINR', 'RM_PX_13_G90', 'RM_PX_13_A90', 'RM_PX_13_KP9', 
    'RM_PX_14', 'RM_PX_14_FSQ', 'RM_PX_14_R', 'RM_PX_14_POS', 'RM_PX_14_RT_M', 'RM_PX_14__RT_PS', 'RM_PX_14_FORM5', 'RM_PX_14_WINR', 'RM_PX_14_G90', 'RM_PX_14_A90', 'RM_PX_14_KP9', 
    'RM_PX_15', 'RM_PX_15_FSQ', 'RM_PX_15_R', 'RM_PX_15_POS', 'RM_PX_15_RT_M', 'RM_PX_15__RT_PS', 'RM_PX_15_FORM5', 'RM_PX_15_WINR', 'RM_PX_15_G90', 'RM_PX_15_A90', 'RM_PX_15_KP9', 
    'RM_PX_16', 'RM_PX_16_FSQ', 'RM_PX_16_R', 'RM_PX_16_POS', 'RM_PX_16_RT_M', 'RM_PX_16__RT_PS', 'RM_PX_16_FORM5', 'RM_PX_16_WINR', 'RM_PX_16_G90', 'RM_PX_16_A90', 'RM_PX_16_KP9', 
    'team_avg_rating', 'team_form_last_5', 'team_win_ratio', 'RM_C_ID', 'RM_C_RT_PS', 'RM_C_FORM5',
    'IS_HOME','H2H_RM_GDIF_L5', 'H2H_PPM_L5', 'H2H_PPM', 'H2H_EXISTS', 'RM_ODD_W','is_real_madrid_win'
]
columns_to_keep_2 = [
    'M_DATE', 'IS_HOME','team_avg_rating', 'team_form_last_5', 'team_win_ratio', 'RM_C_ID', 'RM_C_RT_PS', 
    'RM_C_FORM5', 'RM_G_SCO_L5', 'RM_G_CON_L5', 'RM_GDIF_L5', 'RM_PPM_L5', 'RM_PPM_SEA', 
    'RM_OPP_PPM_L5', 'RM_GPM_VS_TOP', 'RM_PPM_VS_TOP', 'RM_GPM_VS_MID', 'RM_PPM_VS_MID', 
    'RM_GPM_VS_LOW', 'RM_PPM_VS_LOW', 'OP_G_SCO_L5', 'OP_G_CON_L5', 'OP_GDIF_L5', 
    'OP_OPP_POS_L5', 'OP_PPM_L5', 'OP_PPM_SEA', 'OP_GPM_VS_TOP', 'OP_PPM_VS_TOP', 
    'OP_GPM_VS_MID', 'OP_PPM_VS_MID', 'OP_GPM_VS_LOW', 'OP_PPM_VS_LOW', 'OP_G_SCO_ALL', 
    'OP_G_CON_ALL', 'OP_G_SCO_G_CON_RAT', 'H2H_RM_W_L5', 
    'H2H_RM_GDIF_L5', 'H2H_PPM_L5', 'H2H_PPM', 'H2H_EXISTS', 'RM_ODD_W','is_real_madrid_win'
]

# --- KROK 2: Wczytanie i Przygotowanie Danych ---

DATA_PATH = os.path.join(FileUtils.get_project_root(), 'Data','DataSet','real_madrid_master_dataset_win.csv')
df_original = FileUtils.load_csv_safe(DATA_PATH)
info(f"Oryginalny, pełny dataset wczytany. Kształt: {df_original.shape}")

# Konwersja kolumny daty i sortowanie
df_original['M_DATE'] = pd.to_datetime(df_original['M_DATE'])
df_original.sort_values(by='M_DATE', inplace=True)

# Wybór aktywnego zestawu cech
if EXPERIMENT_VERSION == 1:
    columns_to_keep = columns_to_keep_1
    info("Wybrano ZESTAW 1: Pełne dane zawodników.")
    existing_columns = [col for col in columns_to_keep if col in df_original.columns]
    df = df_original[list(dict.fromkeys(existing_columns))].copy()
elif EXPERIMENT_VERSION == 2:
    columns_to_keep = columns_to_keep_2
    info("Wybrano ZESTAW 2: Dane zagregowane i drużynowe.")
    existing_columns = [col for col in columns_to_keep if col in df_original.columns]
    df = df_original[list(dict.fromkeys(existing_columns))].copy()
elif EXPERIMENT_VERSION == 0:
    info("Wybrano ZESTAW 0: Pełny zestaw danych (bez wstępnej selekcji).")
    df = df_original.copy()
else:
    raise ValueError("Nieprawidłowa wartość EXPERIMENT_VERSION. Wybierz 0, 1 lub 2.")

info(f"Wybrano {len(df.columns)} kolumn do eksperymentu. Kształt danych po selekcji: {df.shape}")

# --- KROK 3: Usunięcie Kolumn ID i Informacyjnych (oprócz M_DATE) ---
columns_to_drop = [
    'MATCH_ID', 
    'SEASON', 'OPP_ID', 'RM_C_ID',
    'RM_PX_1','RM_PX_2','RM_PX_3','RM_PX_4','RM_PX_5','RM_PX_6',
    'RM_PX_7','RM_PX_8','RM_PX_9','RM_PX_10','RM_PX_11','RM_PX_12',
    'RM_PX_13','RM_PX_14','RM_PX_15','RM_PX_16',"is_real_madrid_win"]
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# --- KROK 4: Imputacja, Kodowanie ---
info("Rozpoczynam inteligentną imputację i kodowanie...")
pos_cols = [col for col in df.columns if col.startswith('RM_PX_') and col.endswith('_POS')]
if pos_cols:
    df[pos_cols] = df[pos_cols].fillna('NoPos')
player_numeric_cols = [col for col in df.columns if col.startswith('RM_PX_') and not col.endswith('_POS')]
if player_numeric_cols:
    df[player_numeric_cols] = df[player_numeric_cols].fillna(0)
h2h_cols = [col for col in df.columns if col.startswith('H2H_') and col != 'H2H_EXISTS']
if h2h_cols and 'H2H_EXISTS' in df.columns:
    missing_h2h_mask = (df['H2H_EXISTS'] == 0)
    df.loc[missing_h2h_mask, h2h_cols] = df.loc[missing_h2h_mask, h2h_cols].fillna(0)
numeric_opp_cols = ['OP_PPM_SEA', 'OP_G_SCO_ALL', 'OP_G_CON_ALL']
existing_numeric_opp_cols = [col for col in numeric_opp_cols if col in df.columns]
if existing_numeric_opp_cols:
    for col in existing_numeric_opp_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
vs_cols_map = {'OP_GPM_VS_TOP': 'OP_G_SCO_ALL', 'OP_PPM_VS_TOP': 'OP_PPM_SEA', 'OP_GPM_VS_MID': 'OP_G_SCO_ALL', 'OP_PPM_VS_MID': 'OP_PPM_SEA', 'OP_GPM_VS_LOW': 'OP_G_SCO_ALL', 'OP_PPM_VS_LOW': 'OP_PPM_SEA'}
for vs_col, sea_col in vs_cols_map.items():
    if vs_col in df.columns and sea_col in df.columns and df[vs_col].isnull().any():
        df[vs_col] = df[vs_col].fillna(df[sea_col])
l5_cols = [col for col in df.columns if col.startswith('OP_') and col.endswith('_L5')]
if 'OP_PPM_L5' in df.columns and 'OP_PPM_SEA' in df.columns:
    mask_beniaminek_l5 = df['OP_PPM_L5'].isnull()
    if mask_beniaminek_l5.any():
        low_teams_mask = df['OP_PPM_SEA'] < 1.2
        for col in l5_cols:
            if col in df.columns:
                median_for_low_teams = df.loc[low_teams_mask, col].median()
                if pd.isna(median_for_low_teams):
                    median_for_low_teams = df[col].median()
                df.loc[mask_beniaminek_l5, col] = median_for_low_teams
df.fillna(df.median(numeric_only=True), inplace=True)
if pos_cols:
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', dtype=int)
    encoded_data = encoder.fit_transform(df[pos_cols])
    encoded_cols = encoder.get_feature_names_out(pos_cols)
    encoded_df = pd.DataFrame(encoded_data, columns=encoded_cols, index=df.index)
    df = pd.concat([df, encoded_df], axis=1)
    df.drop(columns=pos_cols, inplace=True)
    
expected = [
    'M_DATE','IS_HOME','team_avg_rating','team_form_last_5','team_win_ratio','RM_C_ID','RM_C_RT_PS',
    'RM_C_FORM5','RM_G_SCO_L5','RM_G_CON_L5','RM_GDIF_L5','RM_PPM_L5','RM_PPM_SEA',
    'RM_OPP_PPM_L5','RM_GPM_VS_TOP','RM_PPM_VS_TOP','RM_GPM_VS_MID','RM_PPM_VS_MID',
    'RM_GPM_VS_LOW','RM_PPM_VS_LOW','OP_G_SCO_L5','OP_G_CON_L5','OP_GDIF_L5',
    'OP_OPP_POS_L5','OP_PPM_L5','OP_PPM_SEA','OP_GPM_VS_TOP','OP_PPM_VS_TOP',
    'OP_GPM_VS_MID','OP_PPM_VS_MID','OP_GPM_VS_LOW','OP_PPM_VS_LOW','OP_G_SCO_ALL',
    'OP_G_CON_ALL','OP_G_SCO_G_CON_RAT','H2H_RM_W_L5',
    'H2H_RM_GDIF_L5','H2H_PPM_L5','H2H_PPM','H2H_EXISTS','RM_ODD_W','is_real_madrid_win'
]
missing = [c for c in expected if c not in df.columns]
present = [c for c in expected if c in df.columns]
print("Present:", present)
print("Missing:", missing)


# --- KROK 5: Podział Chronologiczny i Skalowanie ---
TARGET = 'RM_ODD_W'
if TARGET not in df.columns:
    raise ValueError(f"Brak kolumny docelowej '{TARGET}' w zbiorze danych!")
df.sort_values(by='M_DATE', inplace=True)
info("Dane posortowane chronologicznie według M_DATE.")
# Usuwamy M_DATE dopiero teraz, tuż przed podziałem na X i y
if 'M_DATE' in df.columns:
    df.drop(columns=['M_DATE'], inplace=True)

    
X = df.drop(columns=[TARGET])
y = df[TARGET]
# Definiujemy rozmiar zbioru testowego (np. ostatnie 20% danych)
test_size_ratio = 0.2
split_index = int(len(df) * (1 - test_size_ratio))

# Dzielimy dane chronologicznie
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

info("Podzielono dane CHRONOLOGICZNIE na zbiory treningowe i testowe:")
info(f"  - X_train: {X_train.shape}")
info(f"  - X_test:  {X_test.shape}")

# Skalowanie cech numerycznych
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Konwersja z powrotem do DataFrame
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
info("\nSkalowanie danych zakończone. Dane gotowe do modelowania.")

# --- KROK 6: Zapis Danych ---
PROCESSED_DATA_DIR = os.path.join(FileUtils.get_project_root(), 'ProcessedForModeling')
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)
X_train_scaled.to_csv(os.path.join(PROCESSED_DATA_DIR, 'X_train.csv'), index=False)
X_test_scaled.to_csv(os.path.join(PROCESSED_DATA_DIR, 'X_test.csv'), index=False)
y_train.to_csv(os.path.join(PROCESSED_DATA_DIR, 'y_train.csv'), index=False)
y_test.to_csv(os.path.join(PROCESSED_DATA_DIR, 'y_test.csv'), index=False)
joblib.dump(scaler, os.path.join(PROCESSED_DATA_DIR, 'scaler.pkl'))
info(f"Przetworzone dane i scaler zostały zapisane w folderze: {PROCESSED_DATA_DIR}")


INFO: Oryginalny, pełny dataset wczytany. Kształt: (180, 223)
INFO: Wybrano ZESTAW 0: Pełny zestaw danych (bez wstępnej selekcji).
INFO: Wybrano 223 kolumn do eksperymentu. Kształt danych po selekcji: (180, 223)
INFO: Rozpoczynam inteligentną imputację i kodowanie...
Present: ['M_DATE', 'IS_HOME', 'team_avg_rating', 'team_form_last_5', 'team_win_ratio', 'RM_C_RT_PS', 'RM_C_FORM5', 'RM_G_SCO_L5', 'RM_G_CON_L5', 'RM_GDIF_L5', 'RM_PPM_L5', 'RM_PPM_SEA', 'RM_OPP_PPM_L5', 'RM_GPM_VS_TOP', 'RM_PPM_VS_TOP', 'RM_GPM_VS_MID', 'RM_PPM_VS_MID', 'RM_GPM_VS_LOW', 'RM_PPM_VS_LOW', 'OP_G_SCO_L5', 'OP_G_CON_L5', 'OP_GDIF_L5', 'OP_OPP_POS_L5', 'OP_PPM_L5', 'OP_PPM_SEA', 'OP_GPM_VS_TOP', 'OP_PPM_VS_TOP', 'OP_GPM_VS_MID', 'OP_PPM_VS_MID', 'OP_GPM_VS_LOW', 'OP_PPM_VS_LOW', 'OP_G_SCO_ALL', 'OP_G_CON_ALL', 'OP_G_SCO_G_CON_RAT', 'H2H_RM_W_L5', 'H2H_RM_GDIF_L5', 'H2H_PPM_L5', 'H2H_PPM', 'H2H_EXISTS', 'RM_ODD_W']
Missing: ['RM_C_ID', 'is_real_madrid_win']
INFO: Dane posortowane chronologicznie według M_DATE.
I